In [1]:
# !pip install scikeras --quiet
import tensorflow as tf
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

NUM_FEATURES = 31
TIMESTEPS = 1


# from google.colab import drive
# drive.mount('/content/gdrive')

# # X = np.expand_dims(X, 0)
# shape_X = X.shape

2021-11-27 16:33:55.883889: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-27 16:33:55.883923: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


##### Initialise Seed

In [2]:
import numpy as np
import pandas as pd

# random seed for reproducibility
seed = 10
np.random.seed(seed)

# loading of dataset
df = pd.read_csv("https://raw.githubusercontent.com/ehandywhyy/loot-box/main/overall.csv", header=None)
df = pd.read_csv("https://raw.githubusercontent.com/ehandywhyy/loot-box/main/overall.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.142176,0.156880,0.014704,0.127937,0.142299,0.014361,0.155919,0.116134,-0.039784,0.124244,...,0.085967,-0.042296,0.120442,0.209372,0.088930,0.116911,0.165437,0.048526,0.160258,andy
1,0.141793,0.164635,0.022842,0.121625,0.152146,0.030520,0.137039,0.121543,-0.015496,0.104216,...,0.082397,-0.062093,0.101188,0.126066,0.024878,0.098424,0.130661,0.032238,0.173819,andy
2,0.157152,0.147220,-0.009932,0.134796,0.145790,0.010994,0.124892,0.101461,-0.023431,0.112165,...,0.082423,-0.053029,0.108598,0.155843,0.047244,0.118759,0.146550,0.027791,0.173512,andy
3,0.158673,0.142487,-0.016186,0.118908,0.159179,0.040271,0.133175,0.060895,-0.072279,0.106036,...,0.072556,-0.041711,0.101937,0.145692,0.043755,0.096300,0.122580,0.026280,0.190808,andy
4,0.166583,0.157102,-0.009480,0.115094,0.141103,0.026009,0.135121,0.082097,-0.053024,0.128655,...,0.089247,-0.036227,0.114211,0.149136,0.034925,0.121197,0.230113,0.108917,0.143299,andy


##### Check missing values

In [3]:
# print(df.isna().sum())
# # Remove missing values IF AVAILABLE and print first 10 samples
# # df = df.dropna()

##### Divide dataset into X and Y
##### Normalise features within range 0 (minimum) and 1 (maximum)


In [4]:
from sklearn.preprocessing import MinMaxScaler, minmax_scale

dataset = df.values

# divide data into features X and target (Classes) Y
X = dataset[:,0:NUM_FEATURES].astype(float)
Y = dataset[:,NUM_FEATURES]

# check for class imbalance
print(df.groupby(Y).size())

andy     100
azfar    100
chris    100
dtype: int64


In [5]:
from sklearn.preprocessing import OneHotEncoder
from keras.utils.np_utils import to_categorical 

# encode Label for model 
# (Convert target Y to one hot encoded Y for Neural Network)
Y = Y.reshape(-1, 1)
encoder = OneHotEncoder().fit(Y)

# get all the encoded class
print(encoder.get_feature_names_out())

# print X and Y shape
print("X dataset shape: " + str(X.shape))
print("Y dataset shape: " + str(Y.shape))

['x0_andy' 'x0_azfar' 'x0_chris']
X dataset shape: (300, 31)
Y dataset shape: (300, 1)


##### Preparing dataset

In [6]:
from sklearn.model_selection import train_test_split

# split dataset into train and test of 0.8/0.2 ratio
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=seed)

# normalisation to 0 to 1
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# reshaping the dataset to include LSTM
X_train = np.asarray(X_train, dtype=np.float32)
X_train = np.reshape(X_train, (X_train.shape[0], TIMESTEPS, X_train.shape[1]))
X_test = np.asarray(X_test, dtype=np.float32)
X_test = np.reshape(X_test, (X_test.shape[0], TIMESTEPS, X_test.shape[1]))

# converting y data to encoding
y_train = encoder.transform(y_train).toarray()
y_test = encoder.transform(y_test).toarray()

num_classes = y_train.shape[1]

print("X train shape: " + str(X_train.shape))
print("Y train shape: " + str(y_train.shape))
print("X test shape: " + str(X_test.shape))
print("Y test shape: " + str(y_test.shape))

X train shape: (240, 1, 31)
Y train shape: (240, 3)
X test shape: (60, 1, 31)
Y test shape: (60, 3)


### Create Model

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization,\
Flatten, LSTM

def create_model():
    # define model
    model = Sequential()
    model.add(LSTM(units=128, return_sequences=True, 
                 input_shape=(TIMESTEPS,NUM_FEATURES)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(LSTM(units=128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(LSTM(units=64, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    # Softmax for multi-class classification
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    return model

##### Wrap Model in KerasClassifier

In [8]:
# from scikeras.wrappers import KerasClassifier
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=100, 
                            batch_size=10)

### Perform KFold Validation

In [9]:
# from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

num_folds = 10
kfold = KFold(n_splits=num_folds, 
              shuffle=True,
              random_state=seed)

##### Get Accuracy from KFold Validation

In [10]:
results = cross_val_score(model, X_train, y_train, 
                          cv=kfold, error_score="raise", verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
2021-11-27 16:34:24.266848: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (100)
2021-11-27 16:34:24.266894: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-72RQGL3): /proc/driver/nvidia/version does not exist
2021-11-27 16:34:24.267108: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
22/22 [==============================] - 5s 7ms/step - loss: 0.7215 - accuracy: 0.6944
Epoch 2/100
22/22 [==============================] - 0s 7ms/step - loss: 0.2635 - accuracy: 0.8981
Epoch 3/100
22/22 [==============================] - 0s 7ms/step - loss: 0.2411 - accuracy: 0.9259
Epoch 4/100
22/22 [==============================] - 0s 7ms/step - loss: 0.1898 - accuracy: 0.9306
Epoch 5/100
22/22 [==============================] - 0s 7ms/step - loss: 0.3478 - accuracy: 0.8704
Epoch 6/100
22/22 [==============================] - 0s 7ms/step - loss: 0.1905 - accuracy: 0.9306
Epoch 7/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0975 - accuracy: 0.9630
Epoch 8/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1707 - accuracy: 0.9398
Epoch 9/100
22/22 [==============================] - 0s 7ms/step - loss: 0.1661 - accuracy: 0.9352
Epoch 10/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1249 - accuracy: 0.9491
Epoch 11/

22/22 [==============================] - 0s 7ms/step - loss: 0.0772 - accuracy: 0.9769
Epoch 84/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0656 - accuracy: 0.9676
Epoch 85/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0557 - accuracy: 0.9769
Epoch 86/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0431 - accuracy: 0.9769
Epoch 87/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0263 - accuracy: 0.9954
Epoch 88/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0219 - accuracy: 1.0000
Epoch 89/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1388 - accuracy: 0.9676
Epoch 90/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0698 - accuracy: 0.9722
Epoch 91/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0427 - accuracy: 0.9907
Epoch 92/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0302 - accuracy: 0.9861
Epoch 93/100


22/22 [==============================] - 0s 6ms/step - loss: 0.0767 - accuracy: 0.9769
Epoch 65/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0657 - accuracy: 0.9676
Epoch 66/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1119 - accuracy: 0.9491
Epoch 67/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0508 - accuracy: 0.9861
Epoch 68/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0559 - accuracy: 0.9907
Epoch 69/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0695 - accuracy: 0.9815
Epoch 70/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0835 - accuracy: 0.9722
Epoch 71/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1254 - accuracy: 0.9398
Epoch 72/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0541 - accuracy: 0.9722
Epoch 73/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0303 - accuracy: 0.9907
Epoch 74/100


22/22 [==============================] - 0s 6ms/step - loss: 0.0837 - accuracy: 0.9722
Epoch 46/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1241 - accuracy: 0.9583
Epoch 47/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1249 - accuracy: 0.9398
Epoch 48/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1238 - accuracy: 0.9537
Epoch 49/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1129 - accuracy: 0.9583
Epoch 50/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0900 - accuracy: 0.9676
Epoch 51/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0827 - accuracy: 0.9722
Epoch 52/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0833 - accuracy: 0.9583
Epoch 53/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0902 - accuracy: 0.9722
Epoch 54/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0877 - accuracy: 0.9769
Epoch 55/100


...

22/22 [==============================] - 0s 6ms/step - loss: 0.1516 - accuracy: 0.9491
Epoch 17/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0819 - accuracy: 0.9630
Epoch 18/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1006 - accuracy: 0.9676
Epoch 19/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1008 - accuracy: 0.9676
Epoch 20/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1240 - accuracy: 0.9444
Epoch 21/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0977 - accuracy: 0.9537
Epoch 22/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1110 - accuracy: 0.9583
Epoch 23/100
22/22 [==============================] - 0s 9ms/step - loss: 0.0933 - accuracy: 0.9583
Epoch 24/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0745 - accuracy: 0.9769
Epoch 25/100
22/22 [==============================] - 0s 6ms/step - loss: 0.2048 - accuracy: 0.9537
Epoch 26/100


22/22 [==============================] - 0s 6ms/step - loss: 0.0660 - accuracy: 0.9861
Epoch 99/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0881 - accuracy: 0.9769
Epoch 100/100
3/3 [==============================] - 1s 3ms/step - loss: 0.0469 - accuracy: 0.9583
Epoch 1/100
22/22 [==============================] - 4s 6ms/step - loss: 0.7678 - accuracy: 0.7083
Epoch 2/100
22/22 [==============================] - 0s 9ms/step - loss: 0.2492 - accuracy: 0.8981
Epoch 3/100
22/22 [==============================] - 0s 7ms/step - loss: 0.2840 - accuracy: 0.9120
Epoch 4/100
22/22 [==============================] - 0s 7ms/step - loss: 0.1964 - accuracy: 0.9352
Epoch 5/100
22/22 [==============================] - 0s 7ms/step - loss: 0.1301 - accuracy: 0.9491
Epoch 6/100
22/22 [==============================] - 0s 6ms/step - loss: 0.2589 - accuracy: 0.9074
Epoch 7/100
22/22 [==============================] - 0s 6ms/step - loss: 0.2285 - accuracy: 0.9074
Epoch 8/100
22/22 [==

22/22 [==============================] - 0s 7ms/step - loss: 0.0584 - accuracy: 0.9769
Epoch 80/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0494 - accuracy: 0.9907
Epoch 81/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0956 - accuracy: 0.9722
Epoch 82/100
22/22 [==============================] - 0s 7ms/step - loss: 0.1114 - accuracy: 0.9630
Epoch 83/100
22/22 [==============================] - 0s 7ms/step - loss: 0.1300 - accuracy: 0.9676
Epoch 84/100
22/22 [==============================] - 0s 7ms/step - loss: 0.1659 - accuracy: 0.9722
Epoch 85/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0870 - accuracy: 0.9630
Epoch 86/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0752 - accuracy: 0.9722
Epoch 87/100
22/22 [==============================] - 0s 7ms/step - loss: 0.1059 - accuracy: 0.9630
Epoch 88/100
22/22 [==============================] - 0s 8ms/step - loss: 0.0714 - accuracy: 0.9769
Epoch 89/100


22/22 [==============================] - 0s 6ms/step - loss: 0.1304 - accuracy: 0.9722
Epoch 61/100
22/22 [==============================] - 0s 6ms/step - loss: 0.0885 - accuracy: 0.9722
Epoch 62/100
22/22 [==============================] - 0s 9ms/step - loss: 0.0435 - accuracy: 0.9954
Epoch 63/100
22/22 [==============================] - 0s 7ms/step - loss: 0.1216 - accuracy: 0.9444
Epoch 64/100
22/22 [==============================] - 0s 9ms/step - loss: 0.0671 - accuracy: 0.9861
Epoch 65/100
22/22 [==============================] - 0s 6ms/step - loss: 0.1415 - accuracy: 0.9491
Epoch 66/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0469 - accuracy: 0.9861
Epoch 67/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0753 - accuracy: 0.9722
Epoch 68/100
22/22 [==============================] - 0s 9ms/step - loss: 0.0471 - accuracy: 0.9954
Epoch 69/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0935 - accuracy: 0.9676
Epoch 70/100


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  3.5min finished


##### View Model Summary

In [11]:
# model = create_model()


In [12]:
print("Accuracy of %.2f%% (with standard deviation of %.2f%%)" % 
      (results.mean()*100, results.std()*100))

from tensorflow.keras.callbacks import EarlyStopping

# fit the model
es = EarlyStopping(monitor='loss', mode='min', min_delta=0.001, patience=50,
                   verbose=0)
model.fit(X_train, y_train, callbacks=es)

Accuracy of 99.17% (with standard deviation of 1.67%)
Epoch 1/100
24/24 [==============================] - 3s 5ms/step - loss: 0.5642 - accuracy: 0.7792
Epoch 2/100
24/24 [==============================] - 0s 6ms/step - loss: 0.2731 - accuracy: 0.9042
Epoch 3/100
24/24 [==============================] - 0s 6ms/step - loss: 0.1835 - accuracy: 0.9292
Epoch 4/100
24/24 [==============================] - 0s 6ms/step - loss: 0.2065 - accuracy: 0.9250
Epoch 5/100
24/24 [==============================] - 0s 6ms/step - loss: 0.1835 - accuracy: 0.9208
Epoch 6/100
24/24 [==============================] - 0s 5ms/step - loss: 0.2265 - accuracy: 0.9167
Epoch 7/100
24/24 [==============================] - 0s 5ms/step - loss: 0.1301 - accuracy: 0.9417
Epoch 8/100
24/24 [==============================] - 0s 5ms/step - loss: 0.2727 - accuracy: 0.9167
Epoch 9/100
24/24 [==============================] - 0s 5ms/step - loss: 0.0992 - accuracy: 0.9667
Epoch 10/100
24/24 [==============================] - 0

24/24 [==============================] - 0s 6ms/step - loss: 0.0641 - accuracy: 0.9792
Epoch 83/100
24/24 [==============================] - 0s 6ms/step - loss: 0.0396 - accuracy: 0.9875
Epoch 84/100
24/24 [==============================] - 0s 6ms/step - loss: 0.0804 - accuracy: 0.9667
Epoch 85/100
24/24 [==============================] - 0s 6ms/step - loss: 0.1113 - accuracy: 0.9625
Epoch 86/100
24/24 [==============================] - 0s 6ms/step - loss: 0.0809 - accuracy: 0.9792
Epoch 87/100
24/24 [==============================] - 0s 6ms/step - loss: 0.0529 - accuracy: 0.9875
Epoch 88/100
24/24 [==============================] - 0s 6ms/step - loss: 0.1366 - accuracy: 0.9542
Epoch 89/100
24/24 [==============================] - 0s 6ms/step - loss: 0.0680 - accuracy: 0.9792
Epoch 90/100
24/24 [==============================] - 0s 6ms/step - loss: 0.0441 - accuracy: 0.9917
Epoch 91/100
24/24 [==============================] - 0s 8ms/step - loss: 0.0495 - accuracy: 0.9792
Epoch 92/100


In [14]:
# view model summary
model.model.summary()

# save model
# model.model.save(F"/content/gdrive/My Drive/Colab Notebooks/overall_key_classifier.h5")
model.model.save("model/key_classifier.h5")

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_30 (LSTM)              (None, 1, 128)            81920     
                                                                 
 dropout_30 (Dropout)        (None, 1, 128)            0         
                                                                 
 batch_normalization_30 (Bat  (None, 1, 128)           512       
 chNormalization)                                                
                                                                 
 lstm_31 (LSTM)              (None, 1, 128)            131584    
                                                                 
 dropout_31 (Dropout)        (None, 1, 128)            0         
                                                                 
 batch_normalization_31 (Bat  (None, 1, 128)           512       
 chNormalization)                                    

In [15]:
from keras.models import load_model
from sklearn.metrics import accuracy_score, confusion_matrix

# load model
# model.model = load_model("/content/gdrive/My Drive/Colab Notebooks/overall_key_classifier.h5")
# model.model = load_model("model/key_classifier.h5")

y_pred = model.predict(X_test)
y_pred = to_categorical(y_pred)

# evaluate predictions
acc = accuracy_score(y_test, y_pred)
print('Accuracy: %.3f' % acc)

# Making the Confusion Matrix
# import sys
# np.set_printoptions(threshold=sys.maxsize)
# cm = confusion_matrix(y_test, y_pred)
# print(cm)

# from sklearn.metrics import roc_curve
# fpr_RF, tpr_RF, thresholds_RF = roc_curve(df.actual_label.values, df.model_RF.values)
# fpr_LR, tpr_LR, thresholds_LR = roc_curve(df.actual_label.values, df.model_LR.values)

Accuracy: 0.983


In [16]:
# import unseen data to check if model works
new_df = pd.read_csv("https://raw.githubusercontent.com/ehandywhyy/loot-box/main/overall_testa.csv", header=None)

test_row=new_df.iloc[0:17,:-1]
# print("check name")
# print(new_df.iloc[2:3,30:32])
# print(test_row)

# convert to
test = test_row.values.tolist()
test = scaler.transform(test)
test = np.asarray(test, dtype=np.float32)
test = np.reshape(test, (test.shape[0], TIMESTEPS, test.shape[1]))

In [17]:
x_predict = model.predict(test)
x_predict = to_categorical(x_predict)
x_predict = encoder.inverse_transform(x_predict)
print(x_predict)

[['andy']
 ['andy']
 ['andy']
 ['andy']
 ['andy']
 ['chris']
 ['chris']
 ['chris']
 ['chris']
 ['chris']
 ['azfar']
 ['azfar']
 ['azfar']
 ['azfar']
 ['azfar']
 ['chris']
 ['chris']]
